This file shows how good (or bad) I marked the labels.

To check labeling I'm trying to cluster all the abilities
based on the labels they have and check results manually.

## TODO
1. (One day) Write distance function which is going to use nmf classification of abilities texts.

In [1]:
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer

from atod import Abilities

In [2]:
specs = pd.read_csv('../data/702/abilities_specs.csv')
specs.head()

,2_times,3_times,4_times,AbilityBehavior,AbilityCastPoint,AbilityCastRange,AbilityCastRangeBuffer,AbilityChannelTime,AbilityCooldown,AbilityDamage,...,wolf_bat,wolf_damage,wolf_duration,wolf_hp,wolf_index,zombie_interval,ID,damage_per_burn,lvl,per_hit
0,NaN,NaN,NaN,DOTA_ABILITY_BEHAVIOR_PASSIVE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5003.0,0.6,2.5,46.0
1,NaN,NaN,NaN,DOTA_ABILITY_BEHAVIOR_POINT | DOTA_ABILITY_BEH...,0.4,1037.5,NaN,NaN,8.25,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5004.0,NaN,2.5,NaN
2,NaN,NaN,NaN,DOTA_ABILITY_BEHAVIOR_PASSIVE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5005.0,NaN,2.5,NaN
3,NaN,NaN,NaN,DOTA_ABILITY_BEHAVIOR_UNIT_TARGET | DOTA_ABILI...,0.3,600.0,NaN,NaN,70.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5006.0,NaN,2.5,NaN
4,NaN,NaN,NaN,DOTA_ABILITY_BEHAVIOR_NO_TARGET | DOTA_ABILITY...,0.4,NaN,NaN,NaN,13.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,5007.0,NaN,2.5,NaN


In [3]:
columns_to_drop = ['lvl', 'AbilityCastRangeBuffer', 'levelkey', 
                   'HotKeyOverride', 'AbilityTextureName',
                   'LevelsBetweenUpgrades', '_sa_instance_state',
                   'OnCastbar', 'OnLearnbar', 'LinkedSpecialBonus',
                   'ID', 'pk']
specs = specs.drop(columns_to_drop, axis=1)
specs = specs.dropna(axis=1, thresh=2)
specs.shape

(519, 246)

### TODO
write a method to binarize columns by given column name

In [4]:
# encode AbilityBehavior
# get all possible values
behavior = specs.AbilityBehavior
behavior = behavior.str.split('|').apply(lambda x: [y.strip() for y in x])

mlb = MultiLabelBinarizer().fit(behavior)
binary_behavior = pd.DataFrame(mlb.transform(behavior),
                               columns=mlb.classes_).drop([''], axis=1)

X = pd.concat([specs.drop(['AbilityBehavior'], axis=1), binary_behavior], axis=1)

In [5]:
flags = X.AbilityUnitTargetFlags
flags = flags.fillna(value='DOTA_UNIT_TARGET_FLAG_NONE')
flags = flags.str.split('|').apply(lambda x: [y.strip() for y in x])

mlb = MultiLabelBinarizer().fit(flags)
binary_flags = pd.DataFrame(mlb.transform(flags), columns=mlb.classes_)

X = pd.concat([X.drop(['AbilityUnitTargetFlags'], axis=1), binary_flags], axis=1)

In [6]:
# print(set(X.AbilityUnitTargetFlags.values))
target_team = X.AbilityUnitTargetTeam
target_team = target_team.fillna(value='DOTA_UNIT_TARGET_TEAM_NONE')
target_team = target_team.str.split('|').apply(lambda x: [y.strip() for y in x])

mlb = MultiLabelBinarizer().fit(target_team)
binary_team = pd.DataFrame(mlb.transform(target_team), 
                           columns=mlb.classes_)

X = pd.concat([X.drop(['AbilityUnitTargetTeam'], axis=1), binary_team], axis=1)

In [7]:
target_type = X.AbilityUnitTargetType
target_type = target_type.fillna(value='DOTA_UNIT_TARGET_NONE')
target_type = target_type.str.split('|').apply(lambda x: [y.strip() for y in x])

mlb = MultiLabelBinarizer().fit(target_type)
binary_type = pd.DataFrame(mlb.transform(target_type), 
                           columns=mlb.classes_)

X = pd.concat([X.drop(['AbilityUnitTargetType'], axis=1), binary_type], axis=1)

In [8]:
immunity = X.SpellImmunityType
immunity = immunity.fillna(value='SPELL_IMMUNITY_NONE')
immunity = immunity.str.split('|').apply(lambda x: [y.strip() for y in x])

mlb = MultiLabelBinarizer().fit(immunity)
binary_immunity = pd.DataFrame(mlb.transform(immunity), 
                           columns=mlb.classes_)

X = pd.concat([X.drop(['SpellDispellableType'], axis=1), 
               binary_immunity], axis=1)

In [9]:
dispel = X.SpellImmunityType
dispel = dispel.fillna(value='DOTA_UNIT_TARGET_NONE')
dispel = dispel.str.split('|').apply(lambda x: [y.strip() for y in x])

mlb = MultiLabelBinarizer().fit(dispel)
binary_dispel = pd.DataFrame(mlb.transform(dispel), 
                           columns=mlb.classes_)

X = pd.concat([X.drop(['SpellImmunityType'], axis=1), 
               binary_dispel], axis=1)

In [10]:
X['AbilityType'] = X['AbilityType'].apply(
                    lambda x: 1 if x == 'DOTA_ABILITY_TYPE_ULTIMATE' else 0)

X['isUltimate'] = X['AbilityType']
X = X.drop(['AbilityType'], axis=1)

In [11]:
X['AbilityUnitDamageType'] = X['AbilityUnitDamageType'].apply(
                              lambda x: 'DAMAGE_TYPE_NONE' if x is np.NaN else x)

lb = LabelBinarizer().fit(X['AbilityUnitDamageType'])
dmg_type = pd.DataFrame(lb.transform(X['AbilityUnitDamageType']),
                        columns=lb.classes_)

X = pd.concat([X.drop(['AbilityUnitDamageType'], axis=1), dmg_type], axis=1)

In [12]:
X['MaxLevel'] = X['MaxLevel'].fillna(value=4)
X = X.fillna(value=0)

In [13]:
X.shape

(519, 300)

In [17]:
km = KMeans(n_clusters=30, tol=1e-3)
km.fit(X.drop(['name'], axis=1))

ability_by_label = dict()
for name, label in zip(X['name'], km.labels_):
    ability_by_label.setdefault(str(label), [])
    ability_by_label[str(label)].append(name)
    

pprint(json.dumps(ability_by_label, indent=2))

('{\n'
 '  "9": [\n'
 '    "chakram",\n'
 '    "chakram_2"\n'
 '  ],\n'
 '  "12": [\n'
 '    "blink",\n'
 '    "enfeeble",\n'
 '    "blood_bath",\n'
 '    "rupture",\n'
 '    "fissure",\n'
 '    "torrent",\n'
 '    "replicate",\n'
 '    "toss",\n'
 '    "frost_armor",\n'
 '    "tricks_of_the_trade",\n'
 '    "shrapnel",\n'
 '    "fatal_bonds",\n'
 '    "upheaval",\n'
 '    "rain_of_chaos",\n'
 '    "blink",\n'
 '    "life_drain",\n'
 '    "phantom_strike",\n'
 '    "psionic_trap",\n'
 '    "weave",\n'
 '    "track",\n'
 '    "time_lapse",\n'
 '    "ice_path",\n'
 '    "macropyre",\n'
 '    "ice_vortex",\n'
 '    "cold_snap",\n'
 '    "true_form_battle_cry",\n'
 '    "phantasm",\n'
 '    "glimpse",\n'
 '    "mystic_flare",\n'
 '    "stone_caller",\n'
 '    "splinter_blast",\n'
 '    "cold_embrace",\n'
 '    "boundless_strike",\n'
 '    "tree_dance"\n'
 '  ],\n'
 '  "6": [\n'
 '    "hookshot"\n'
 '  ],\n'
 '  "2": [\n'
 '    "silence",\n'
 '    "marksmanship",\n'
 '    "shadowraze2",\n'
